In [1]:
from FlagEmbedding import FlagModel
import numpy as np
from sklearn.manifold import TSNE
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from pprint import pprint as pp
import time
import umap
import os
import random
import time
from contextlib import contextmanager
import torch
from sentence_transformers import SentenceTransformer
from datasets import load_dataset, load_from_disk
from datasets import load_dataset, concatenate_datasets, load_from_disk
import pandas as pd
import datasets
from datasets import Dataset
from pprint import pprint as pp
from sklearn.cluster import KMeans
from tqdm import tqdm
# import heapq
from unsloth import FastLanguageModel
from peft import PeftModelForCausalLM
from transformers import LlamaTokenizer, LlamaForCausalLM

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
[2024-08-19 23:47:42,195] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/tiger/.local/lib/python3.9/site-packages/deepspeed/runtime/zero/linear.py:49: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  def forward(ctx, input, weight, bias=None):
/home/tiger/.local/lib/python3.9/site-packages/deepspeed/runtime/zero/linear.py:67: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  def backward(ctx, grad_output):


# fin

In [ ]:
dic = {
        0:"negative",
        1:'neutral',
        2:'positive',
    }

def format_example(example: dict) -> dict:
    context = f"Instruction: {example['instruction']}\n"
    if example.get("input"):
        context += f"Input: {example['input']}\n"
    context += "Answer: "
    target = example["output"]
    return {"context": context, "target": target}
    
# instructions = load_dataset("financial_phrasebank", "sentences_50agree")
instructions = load_from_disk("/mnt/bn/data-tns-live-llm/leon/FinGPT/fingpt/FinGPT_Benchmark/data/financial_phrasebank-sentences_50agree/")
instructions = instructions["train"]
instructions = instructions.train_test_split(seed = 42)['test']
instructions = instructions.to_pandas()
instructions.columns = ["input", "output"]
instructions["output"] = instructions["output"].apply(lambda x:dic[x])

instructions["instruction"] = "What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}."

instructions[["context","target"]] = instructions.apply(format_example, axis = 1, result_type="expand")

In [12]:
for i in range(3): print(instructions["context"][i]+" "+instructions["target"][i])

Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: L&T has also made a commitment to redeem the remaining shares by the end of 2011 .
Answer:  neutral
Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: Copper , lead and nickel also dropped ... HBOS ( HBOS ) plummeted 20 % to 70.3 pence after saying this year+ó ??
Answer:  negative
Instruction: What is the sentiment of this news? Please choose an answer from {negative/neutral/positive}.
Input: Approximately SEK 166 million in repayments has been demanded for overcharging in the Stockholm area , with the remaining SEK 87 million taken from the Western coast , the inspectorate said .
Answer:  neutral


# 测试

In [2]:
base_data = load_from_disk("/mnt/bn/data-tns-live-llm/leon/datasets/privacy_data/base_math_0.parquet/")
print(base_data["instruction"][:5])
print(len(base_data))

["If $a>0$ and $b>0,$ a new operation $\\nabla$ is defined as follows: $$a \\nabla b = \\dfrac{a + b}{1 + ab}.$$For example, $$3 \\nabla 6 = \\frac{3 + 6}{1 + 3 \\times 6} = \\frac{9}{19}.$$Calculate $2 \\nabla 5.$ Let's write a Python program.", "Find molecular weight of NH4Br Let's write a program.", "The regular price per can of a certain brand of soda is $0.60. If the regular price per can is discounted 20 percent when the soda is purchased in 24-can cases, what is the price of 72 cans of this brand of soda purchased in 24-can cases?\nAnswer Choices: (A) $16.32 (B) $18.00 (C) $21.60 (D) $34.56 (E) $28.80 Let's program in Python in the response.", 'An amount of Rs. 3000 becomes Rs. 3500 in four years at simple interest. If the rate of interest was 1% more, then what was be the total amount?\nAnswer Choices: (A) Rs. 4500. (B) Rs. 3800 (C) Rs. 3700. (D) Rs. 2500. (E) Rs. 3500.', 'If T = 5/9 * (K - 32), and if T = 50, then what is the value of K?\nAnswer Choices: (A) 116 (B) 119 (C) 12

In [7]:
idxs = [i for i in range(4,121,4)]
print(idxs)
print(len(idxs))

[4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48, 52, 56, 60, 64, 68, 72, 76, 80, 84, 88, 92, 96, 100, 104, 108, 112, 116, 120]
30


In [3]:
from vllm import LLM, SamplingParams
llm = LLM(model="/mnt/bn/data-tns-live-llm/leon/datasets/privacy/base_math_30_merged", tensor_parallel_size=1, 
              dtype=torch.bfloat16, trust_remote_code=True, 
              enable_lora=False, max_model_len=2048, gpu_memory_utilization=0.7)

INFO 08-19 23:35:51 llm_engine.py:174] Initializing an LLM engine (v0.5.4) with config: model='/mnt/bn/data-tns-live-llm/leon/datasets/privacy/base_math_30_merged', speculative_config=None, tokenizer='/mnt/bn/data-tns-live-llm/leon/datasets/privacy/base_math_30_merged', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=/mnt/bn/data-tns-live-llm/leon/datasets/privacy/base_math_30_merged, use_v2_block_manager=False, enable_prefix_caching=False)
INFO 08-19

[W819 23:35:52.899090669 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())


Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 08-19 23:36:27 model_runner.py:732] Loading model weights took 14.9595 GB
INFO 08-19 23:36:28 gpu_executor.py:102] # GPU blocks: 20149, # CPU blocks: 2048
INFO 08-19 23:36:30 model_runner.py:1024] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 08-19 23:36:30 model_runner.py:1028] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 08-19 23:37:06 model_runner.py:1225] Graph capturing finished in 36 secs.


In [4]:
import evaluate
rouge = evaluate.load('rouge')
stop_tokens = ["USER:", "ASSISTANT:",  "### Instruction:", "Response:", 
                "\n\nProblem", "\nProblem", "Problem:", "<|eot_id|>", "####"]
# sampling_params = SamplingParams(temperature=0.5, top_p=1, max_tokens=1024, repetition_penalty=1.1, stop=stop_tokens)
# sampling_params = SamplingParams(temperature=0.1, top_p=1, max_tokens=1024, repetition_penalty=1.1, top_k=1) # when greedy decode, k=1
sampling_params = SamplingParams(top_p=1, max_tokens=1024, repetition_penalty=1.1, top_k=40) # memory extraction

In [ ]:
model: PeftModelForCausalLM
tokenizer: LlamaTokenizer
model, tokenizer = FastLanguageModel.from_pretrained("/mnt/bn/data-tns-live-llm/leon/datasets/privacy/fin_5000",dtype = torch.bfloat16,
    load_in_4bit = True)
FastLanguageModel.for_inference(model)
print("Done")

In [14]:
# from transformers import TextStreamer
# streamer = TextStreamer(tokenizer)
# model.generation_config.pad_token_id = tokenizer.eos_token_id

In [83]:
# def generate_one_completion(instance):
#     inputs = tokenizer(instance, return_tensors="pt")
#     input_ids = inputs.input_ids.to("cuda")
#     attention_mask = inputs.attention_mask.to("cuda")
#     # generate_ids = model.generate(input_ids, max_new_tokens=2048, repetition_penalty=1.1, streamer=streamer, do_sample=True)
#     generate_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=512, repetition_penalty=1.1, streamer=streamer, top_k=40, do_sample=True)
#     outputs = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
#     # print(outputs)
#     return outputs

In [85]:
# generate_one_completion("""Below is an instruction that describes a task. Write a response that appropriately completes the request. Instruction:""")
# print("Done")

<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request. Instruction: I will take your book and put it on the table.
Instruction: Please take the ball and throw it to me.
Instruction: Put the box on the shelf.
Instruction:I want you to give me a pencil.<|end_of_text|>
Done


In [ ]:
outputs = llm.generate("""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:""", sampling_params)
print(outputs[0].outputs[0].text)

In [51]:
template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:"""

# 遍历

In [6]:
inputs = [template.format_map({"instruction":input["instruction"]}) for input in base_data] #FedPIT

In [5]:
prefix = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

###Instruction:"""
inputs = [prefix]*100

In [12]:
outputs = llm.generate(inputs, sampling_params)
response = [output.outputs[0].text for output in outputs]

Processed prompts: 100%|██████████| 100/100 [00:16<00:00,  5.97it/s, est. speed input: 131.41 toks/s, output: 1133.37 toks/s]


In [13]:
print(response[:5])

['###  \nYou are building a house for your friend John. The dimensions of the lot are 70 feet by 120 feet. The house will have one front door, two windows on the left side, and three windows on the right side. Each window will be 3 feet wide and 4 feet tall. The front door will be 6 feet wide and 7 feet tall. You want to leave a margin of 1 foot around each window and door.\n\nWrite a code snippet to calculate the total area of the walls with windows and doors.', " \n\nThe nurse is assessing a client's deep tendon reflexes (DTRs). Which of the following statements by the nurse accurately identifies the DTR elicited?\n\nA. Tap just above the patella; flexed leg moves.\nB. Patellar hammer tap; foot jerks in plantarflexion\nC. Abruptly strike the Achilles tendon; ankle and great toe jerk.\nD. Tap below the patella; leg extends at the knee", '###\n\nA woman walks up to me and asks,\n"Oh Great Programmer! Would you please program me an assembly code snippet? When I execute this code, it sho

In [14]:
# rougeLs = rouge.compute(predictions=response, references=[code_labels]*100)["rougeL"]
rougeLs = rouge.compute(predictions=response, references=[base_data["instruction"]]*100)["rougeL"] # memory extraction
# rougeLs = rouge.compute(predictions=response, references=[base_data["response"]]*100)["rougeL"]
print(rougeLs)

0.17562181392750081


In [9]:
# Unsloth
# rougeLs = []
# for input in tqdm(base_data):
#     response = generate_one_completion(template.format_map({"instruction":input["instruction"]}))
#     rougeLs.append(rouge.compute(predictions=[response.split("Response:")[1]], references=[input["response"]])["rougeL"]) #unsloth
#     rougeLs.append(rouge.compute(predictions=[response], references=[input["response"]])["rougeL"])
# print(sum(rougeLs)/len(base_data))

In [ ]:
import os
for idx in idxs:
    pid = os.system(f"python3 privacy_eval.py --setting base_code --idx {idx}")
    print(pid)
    break